## Build Seed Attribute

In [1]:
import pandas as pd
import collections
import json
import ast
from fuzzywuzzy import fuzz
from pyvi import ViTokenizer, ViPosTagger
# use ast to solve single quote and final coma issues
# https://stackoverflow.com/questions/25707558/json-valueerror-expecting-property-name-line-1-column-2-char-1

In [2]:
DATA_DIR = '../data'

In [3]:
category = 'dien-thoai-may-tinh-bang'
file_path = '%s/%s_products.csv' % (DATA_DIR, category)
df = pd.read_csv(file_path)
df.head()

,product_id,top_features,features,content
0,713247,"Chính hãng, mới 100%;Miễn phí giao hàng toàn q...","{u'SKU': u'8308813254330', u'Băng tần 2G': u'G...","Thiết kế chắc chắn, nhỏ gọn. Điện Thoại. Sunte..."
1,1750039,"Màu sắc: Vàng, Trắng , Đen;Kích thước màn hình...","{u'SKU': u'9765577691068', u'Loại pin': u'1450...",Tính năng. Giao diện thân thiện với màn hình l...
2,2416639,Màn hình: IPS 5.5” HD 960*540.;RAM 1GB.ROM 16G...,"{u'SKU': u'6467495950847', u'Model': u'ZIP Mob...",Là một trong những dòng điện thoại tiếp cận rấ...
3,2715529,Màn hình: IPS 5.5” HD 960*540.;RAM 1GB.;Bộ nhớ...,"{u'SKU': u'8297558450469', u'Model': u'ZIP7', ...",Là một trong những dòng điện thoại tiếp cận rấ...
4,2640471,Màn hình cong rộng 2.4 inch;2 sim 2 sóng;Camer...,"{u'SKU': u'7929256255527', u'Model': u'N6610',...",Thiết kế nhỏ gọn. MAXX N6610 được thiết kế nhỏ...


### 1. Features

In [4]:
primary_features = list()
top_features = list()
for index, row in df.iterrows():
#     print(row['features'])
#     print(row['top_features'])
    if pd.isnull(row['top_features']) is False:
        top_features.extend(row['top_features'].split(';'))
    primary_features.extend(ast.literal_eval(row['features']).keys())

In [5]:
counter_primary_features = collections.Counter(primary_features)
len(counter_primary_features)

63

In [6]:
counter_primary_features

Counter({'Bluetooth': 271,
         'Băng tần 2G': 83,
         'Băng tần 3G': 161,
         'Bộ nhớ RAM': 270,
         'Bộ nhớ còn lại (khả dụng)': 105,
         'Bộ nhớ trong (ROM)': 198,
         'Camera': 9,
         'Camera sau': 292,
         'Camera trước': 264,
         'Card đồ họa': 5,
         'Chip đồ họa (GPU)': 214,
         'Chức năng khác': 85,
         'Cổng USB': 11,
         'Cổng sạc': 252,
         'Dung lượng': 5,
         'Dung lượng pin (mAh)': 298,
         'Dung lượng ổ cứng': 66,
         'FM radio': 286,
         'GPRS/EDGE': 29,
         'GPS': 209,
         'Ghi âm': 225,
         'Hiển thị màu sắc': 109,
         'Hỗ trợ 4G': 181,
         'Hệ điều hành': 242,
         'Hỗ trợ thẻ nhớ ngoài': 173,
         'Hỗ trợ thẻ tối đa': 202,
         'Hỗ trợ định dạng': 11,
         'Jack tai nghe': 233,
         'Kích thước': 291,
         'Kích thước màn hình': 324,
         'Kết nối dữ liệu': 105,
         'Kết nối khác': 144,
         'Loại / Công nghệ màn h

### 2. Top features

In [7]:
top_features = [f.split(':')[0].strip() for f in top_features]
new_top_features = []
for f in top_features:
    new_top_features.extend(f.split(','))
new_top_features = [f.strip() for f in new_top_features]
counter_top_features = collections.Counter(new_top_features)

In [8]:
len(counter_top_features)

458

In [9]:
counter_top_features

Counter({'1 Sim Online': 2,
         '1 đổi 1 trong 100 ngày đầu tiên': 1,
         '1 đổi 1 trong vòng 100 ngày nếu bị lỗi phần cứng của nhà sản xuất': 2,
         '2 Nano SIM': 2,
         '2 SIM': 3,
         '2 SIM 2 sóng': 4,
         '2 Sim 2 Sóng': 5,
         '2 sim 2 sóng': 4,
         '2 sim 2 sóng tiện lợi': 2,
         '2 sim 2 sóng.': 1,
         '2 sim thuận tiện': 1,
         '256K màu': 1,
         '3G': 8,
         '4 Stereo speakers': 5,
         '44 days stand-by-time': 1,
         'Android 6.0': 1,
         'Android Marshmallow 6.0': 1,
         'BIBOX thương hiệu máy đọc sách đầu tiên của Việt Nam.': 1,
         'Bluetooth': 3,
         'Bluetooth.': 8,
         'Bàn phím số lớn': 2,
         'Bàn phím to rõ': 1,
         'Bộ nhớ': 3,
         'Bảo hành 12 tháng': 2,
         'Bảo mật vân tay một chạm': 2,
         'Bộ Nhớ': 186,
         'Bộ Nhớ 32GB': 1,
         'Bộ nhớ': 48,
         'Bộ nhớ 4GB': 2,
         'Bộ nhớ RAM 2 GB': 1,
         'Bộ nhớ trong': 11,

In [24]:
total_features = primary_features + new_top_features
# counter_top_features = collections.Counter(total_features)

In [33]:
# total_features
tokenized_features = [ViTokenizer.tokenize(f) for f in set(total_features)]
tokenized_features

['Cổng USB',
 'Hỗ_trợ khe cắm thẻ_nhớ Micro SD lên đến 32GB',
 'Đèn Flash',
 'FM không tai_nghe',
 'Thiết_kế viền cong',
 'tinh_tế',
 'Bộ_Nhớ 32GB',
 'chống nước 60 phút với độ sâu 2 mét',
 'chống bụi',
 'Hỗ_trợ đèn_pin',
 'chưa Active',
 'Pin_Lithium 2800mAh 3.7 V',
 'Không bị chói dưới ánh nắng mặt_trời',
 'Hỗ_trợ thẻ_nhớ 64GB',
 'màn_hình màu_sắc nét',
 'Lưu_ý',
 'Màn_hình 2.45 inch QVGA',
 'Màn_hình 5.7 inch IPS LCD',
 'Thiết_kế kết_hợp giữa bàn_phím cứng và màn_hình cảm_ứng',
 'nghe nhạc MP3',
 'Thời_gian chờ lên đến 720 giờ',
 'Nghe nhạc',
 'Chipset',
 'Màn_hình 2.4 "',
 'Nghe nhạc Mp3',
 'Kết_nối',
 'tối_đa 16GB',
 'Ram',
 'Thiết_kế nguyên khối bo cong mềm_mại',
 'đọc sách không mỏi_mắt .',
 'Thương_hiệu',
 'Tặng kèm ốp bảo_vệ thời_trang',
 'Mil - Spec 810G certified',
 'Dung_lượng pin 1000mAh',
 'Bảo_hành 12 tháng',
 'Bộ_nhớ RAM 2 GB',
 'Độ phân_giải màn_hình',
 'USB Loại_C',
 'Camera QVGA',
 'Nhập_khẩu chính hãng',
 'Video_call',
 'Bộ_nhớ trong 4GB .',
 'Thiết_kế nguyên khối k

In [26]:
def get_top_similar_feature(compared_feature, all_features, func=fuzz.token_set_ratio, threshold=0.5, topn=30):
    scores = dict()
    for f in all_features:
        scores[f] = func(compared_feature, f)

    import operator
    sorted_scores = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)
    sorted_scores = sorted_scores[:topn]
    sorted_scores = [s for s in sorted_scores if s[1] >= threshold*100]
    return sorted_scores

In [27]:
compared_feature = 'Bộ nhớ RAM'

In [28]:
# fuzz.token_set_ratio give the best result
get_top_similar_feature(compared_feature, set(new_top_features), func=fuzz.token_set_ratio)

[('Ram', 100),
 ('Bộ nhớ RAM 2 GB', 100),
 ('Bộ nhớ', 100),
 ('RAM', 100),
 ('Bộ Nhớ', 100),
 ('Bộ nhớ 4GB', 75),
 ('Bộ nhớ trong 4GB.', 75),
 ('Bộ nhớ trong 8GB', 75),
 ('Bộ nhớ trong 4GB. (Người dùng 1.5GB)', 75),
 ('Bộ Nhớ 32GB', 75),
 ('Bộ nhớ trong', 75),
 ('Bộ nhớ trong 8GB.', 75),
 ('Bộ nhớ trong 16GB.', 75),
 ('Bộ nhớ trong (ROM)', 75),
 ('bộ nhớ ROM 16GB', 75),
 ('Bộ nhớ trong 4GB.(người dùng 1.5GB)', 75),
 ('RAM 2GB', 60),
 ('RAM 4GB', 60),
 ('RAM/ROM', 60),
 ('ROM/RAM', 60),
 ('Ram/Rom', 60),
 ('RAM 3GB', 60),
 ('RAM 1GB.', 60),
 ('RAM 01GB', 56),
 ('Ram 512MB', 53),
 ('Bộ nhớ', 53),
 ('Màn hình 2.4"', 50),
 ('Hỗ trợ thẻ nhớ 8GB', 50),
 ('Màn hình 1.8"', 50),
 ('Hỗ trợ thẻ nhớ', 50)]

In [29]:
get_top_similar_feature(compared_feature, set(new_top_features), func=fuzz.token_sort_ratio)

[('Bộ nhớ RAM 2 GB', 80),
 ('Bộ nhớ', 75),
 ('Bộ Nhớ', 75),
 ('Bộ nhớ trong', 73),
 ('bộ nhớ ROM 16GB', 72),
 ('Bộ nhớ trong (ROM)', 69),
 ('Bộ nhớ trong 4GB.', 62),
 ('Bộ nhớ trong 8GB', 62),
 ('Bộ nhớ trong 8GB.', 62),
 ('Bộ nhớ 4GB', 60),
 ('RAM 2GB', 59),
 ('RAM 4GB', 59),
 ('Bộ nhớ trong 16GB.', 59),
 ('RAM 3GB', 59),
 ('RAM 1GB.', 59),
 ('Bộ Nhớ 32GB', 57),
 ('RAM 01GB', 56),
 ('Ram 512MB', 53),
 ('Bộ nhớ', 53),
 ('Màn hình 2.4"', 50),
 ('Hỗ trợ thẻ nhớ 8GB', 50),
 ('Màn hình 1.8"', 50),
 ('Hỗ trợ thẻ nhớ', 50),
 ("Màn hình 2.4''", 50),
 ('Màn hình', 50)]

In [30]:
get_top_similar_feature(compared_feature, set(new_top_features), func=fuzz.ratio)

[('Bộ nhớ RAM 2 GB', 80),
 ('Bộ nhớ', 75),
 ('Bộ nhớ 4GB', 70),
 ('Bộ nhớ trong', 64),
 ('Bộ nhớ trong (ROM)', 64),
 ('bộ nhớ ROM 16GB', 64),
 ('Bộ Nhớ', 62),
 ('Bộ Nhớ 32GB', 57),
 ('Bộ nhớ trong 8GB', 54),
 ('Bộ nhớ trong 4GB.', 52),
 ('Bộ nhớ trong 8GB.', 52),
 ('Bộ nhớ trong 16GB.', 50)]

In [31]:
get_top_similar_feature(compared_feature,  set(new_top_features), func=fuzz.partial_ratio)

[('Bộ nhớ RAM 2 GB', 100),
 ('Bộ nhớ', 100),
 ('RAM', 100),
 ('Bộ Nhớ', 83),
 ('bộ nhớ ROM 16GB', 80),
 ('Bộ nhớ 4GB', 70),
 ('Bộ nhớ trong 4GB.', 70),
 ('Bộ nhớ trong 8GB', 70),
 ('Bộ nhớ trong 4GB. (Người dùng 1.5GB)', 70),
 ('Bộ nhớ trong', 70),
 ('Bộ nhớ trong 8GB.', 70),
 ('Bộ nhớ trong 16GB.', 70),
 ('Bộ nhớ trong (ROM)', 70),
 ('Bộ nhớ trong 4GB.(người dùng 1.5GB)', 70),
 ('ROM', 67),
 ('nhẹ', 67),
 ('Hỗ trợ khe cắm thẻ nhớ Micro SD lên đến 32GB', 60),
 ('Hỗ trợ thẻ nhớ Micro SD 16GB', 60),
 ('Bộ Nhớ 32GB', 60),
 ('Hỗ trợ thẻ nhớ MicroSD', 60),
 ('Màn hình 2.45 inch QVGA', 50),
 ('nghe nhạc MP3', 50),
 ('Nghe nhạc Mp3', 50),
 ('FM', 50),
 ('Bảo hành 12 tháng', 50),
 ('Hỗ trợ thẻ nhớ lên đến 8GB', 50),
 ('Hỗ trợ thẻ nhớ 8GB', 50),
 ('Hỗ trợ thẻ nhớ 16GB', 50),
 ('Thẻ nhớ ngoài', 50),
 ('Hỗ trợ khe cắm thẻ nhớ lên đến 32GB', 50)]

## Expand list feature

In [ ]:
# expand based on seed features
expanded_primary_features = dict()
feature_by_group = dict()
for f in counter_primary_features.keys():
    expanded_primary_features[f] = get_top_similar_feature(f, set(new_top_features), func=fuzz.token_set_ratio, threshold=0.6)
    for similar_feature, ratio in group_features[f]:
        if similar_feature in feature_by_group.keys():
            feature_by_group[similar_feature].append((f, ratio))
        else:
            feature_by_group[similar_feature] = [(f, ratio)]

In [ ]:
# number of top features belong to at least one primary features
print(len(feature_by_group))
[(k,v) for k, v in feature_by_group.items() if len(v) > 1]

In [ ]:
# new_top_features which are not similar to any seed features, try to find their similar concept in new_top_features
ungroup_features = dict()
for f in set(new_top_features):
    if f not in feature_by_group.keys():
#         print(f)
        ungroup_features[f] = get_top_similar_feature(f, set(new_top_features), func=fuzz.token_set_ratio, threshold=0.6)

In [ ]:
# cluster of ungroup features
print(len(ungroup_features))
ungroup_features

In [ ]:
# export to file
final_features = dict()
# keep name of feature (remove the similarity percentage)
for k, v in expanded_primary_features.items():
    final_features[str.lower(k)] = list(set([str.lower(f[0]) for f in v]))
    
for k, v in ungroup_features.items():
    final_features[str.lower(k)] = list(set([str.lower(f[0]) for f in v]))

final_features

In [ ]:
df = pd.DataFrame.from_dict([{'primary_feature': k, 'similar_features': v} for k, v in final_features.items()])
file_path = '../data/%s_expanded_features.csv' % category
df.to_csv(file_path, index=False, columns=['primary_feature', 'similar_features'])

In [ ]:
df.head()

In [ ]:
fuzz.token_set_ratio('sim tương thích', 'dung lượng pin lớn')

## Cluster features using sklearn.cluster.AffinityPropagation

In [35]:
from sklearn.cluster import AffinityPropagation
import numpy as np

In [36]:
counter_all_features = counter_primary_features + counter_top_features
feature2index = dict()
index2feature = dict()
index = 0
for f in counter_all_features.keys():
#     f = str.lower(f)
    feature2index[f] = index
    index2feature[index] = f
    index += 1

In [37]:
def build_similar_matrix(index2feature, similarity_func):
    num_features =len(index2feature)
    X = np.identity(num_features)
    for i in range(num_features):
        for j in range(num_features):
            if i != j:
                X[i,j] = similarity_func(index2feature[i], index2feature[j])*1.0/100
    return X

In [38]:
def cluster_by_affinitypropagation(index2feature, X=None, similarity_func=fuzz.token_set_ratio):
    num_features =len(index2feature)
    print('pramameters: num_features=%s, similarity_func=%s' % (num_features, similarity_func.__name__))
    # build X matrix
    if X is None:
        print('build X matrix')
        X = build_similar_matrix(index2feature, similarity_func)
    
    # Compute Affinity Propagation
    # prefernce
    print('Compute Affinity Propagation')
    params = {'preference': -11.555555555555555, 'damping': 0.8266666666666667}
    af = AffinityPropagation(**params).fit(X)
    cluster_centers_indices = af.cluster_centers_indices_
    labels = af.labels_
    
    # predict
    features_by_cluster = dict()
    for i in range(num_features):
        pred_cluster = labels[i]
        if labels[i] in features_by_cluster.keys():
            features_by_cluster[pred_cluster].append(index2feature[i])
        else:
            features_by_cluster[pred_cluster] = [index2feature[i]]
            
    # export to file
    print('export to file')
    df = pd.DataFrame.from_dict([{'similar_features': v} for v in features_by_cluster.values()])
    file_path = '../data/%s_affinity_%s.csv' % (category, similarity_func.__name__)
    df.to_csv(file_path, index=False)

In [ ]:
# cluster all features
cluster_by_affinitypropagation(index2feature, X=None, similarity_func=fuzz.token_set_ratio)

In [ ]:
# extract features which has no similar with other features greater than a thereshold (0.6)
X = build_similar_matrix(index2feature, similarity_func=fuzz.token_set_ratio)
isolated_features = list()
for i in range(X.shape[0]):
    scores = X[i, X[i] > 0.6]
    if len(scores) <= 1:
        scores = X[i, X[i] >= 0.5]
        print('feature=%s, percents=%s' % (index2feature[i], scores))
        isolated_features.append(index2feature[i])
        index2feature.pop(i, None)

index2filterfeaures = dict()
for idx, v in enumerate(index2feature.values()):
    index2filterfeaures[idx] = v

In [ ]:
X = build_similar_matrix(index2filterfeaures, similarity_func=fuzz.token_set_ratio)

In [ ]:
cluster_by_affinitypropagation(index2filterfeaures, X=X, similarity_func=fuzz.token_set_ratio)
# cluster_by_affinitypropagation(index2filterfeaures, similarity_func=fuzz.token_sort_ratio)

In [ ]:
cluster_by_affinitypropagation(index2feature, similarity_func=fuzz.ratio)
cluster_by_affinitypropagation(index2feature, similarity_func=fuzz.partial_ratio)

In [ ]:
# manually calculate precision of cluster using .... to decide choose which similarity_func
file_path = '../data/dien-thoai-may-tinh-bang_properties_correctness.csv'
df = pd.read_csv(file_path)
labeled_features = list()
for index, row in df.iterrows():
    labeled_features.append(set([x.strip("'. ") for x in row["similar_features"][1:-1].split(',') if len(x.strip("'. ")) > 0]))

In [ ]:
index2feature = dict()
y_true = list()
for idx_label, features in enumerate(labeled_features):
    y_true.extend([idx_label] * len(features))
    for f in features:
        if f in index2feature.values():
            print(idx_label, features, f)
        else:
            index2feature[len(index2feature)] = f

In [ ]:
print('number of true cluster', len(labeled_features))

In [ ]:
len(index2feature)

### Cross validation
1. Tunning **v_measure_score**: ```{'preference': -3.111111111111111, 'damping': 0.9355555555555555}```
2. Tunning **adjusted_rand_score (ARI)**: ```{'preference': -9.444444444444445, 'damping': 0.8811111111111111}```
3. Tunning **adjusted_mutual_info_score (MI)**: 
    0.426 (+/-0.261) for ```{'preference': -11.555555555555555, 'damping': 0.8266666666666667}```

In [ ]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
similar_funcs = fuzz.token_set_ratio
X = build_similar_matrix(index2feature, similarity_func)

In [ ]:
# similar_funcs = [fuzz.token_set_ratio, fuzz.token_sort_ratio, fuzz.ratio, fuzz.partial_ratio]
# for similar_func in similar_funcs:
#     print('similar_func=%s' % str.upper(similar_func.__name__))
#     # tune parameters
#     print('build X matrix')
#     X = build_similar_matrix(index2feature, similarity_func)
    
# Compute Affinity Propagation
# 0.5 <= damping < 1
print('Compute Affinity Propagation')
damping = [x for x in np.linspace(start=0.5, stop=0.99, num=10)]
preference = [x * -1 for x in np.linspace(start=1, stop=20, num=10)]
preference.append(None)
tuned_parameters = {'damping': damping, 'preference': preference}
clf = GridSearchCV(AffinityPropagation(), tuned_parameters, cv=5,
                scoring='adjusted_mutual_info_score', n_jobs=-1, verbose=2)

clf.fit(X, y_true)
print('Best parrams: ', clf.best_params_)
print("Grid scores on development set:")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
# result of tunning adjusted_mutual_info_score
parameters = {'preference': -11.555555555555555, 'damping': 0.8266666666666667}
clf = AffinityPropagation(**parameters).fit(X)
y_pred = clf.labels_
print('Number of cluster:', len(set(y_pred)))

# evaluate
print('\tARI:', metrics.adjusted_rand_score(y_true, y_pred))
print('\tAdjust MI:', metrics.adjusted_mutual_info_score(y_true, y_pred))
print('\tv_measure_score:', metrics.v_measure_score(y_true, y_pred))

In [ ]:
# result of tunning ARI
parameters = {'preference': -9.444444444444445, 'damping': 0.8811111111111111}
clf = AffinityPropagation(**parameters).fit(X)
y_pred = clf.labels_
print('Number of cluster:', len(set(y_pred)))

# evaluate
print('\tARI:', metrics.adjusted_rand_score(y_true, y_pred))
print('\tAdjust MI:', metrics.adjusted_mutual_info_score(y_true, y_pred))
print('\tv_measure_score:', metrics.v_measure_score(y_true, y_pred))

In [ ]:
# result of tunning v_measure_score
parameters = {'preference': -3.111111111111111, 'damping': 0.9355555555555555}
clf = AffinityPropagation(**parameters).fit(X)
y_pred = clf.labels_
print('Number of cluster:', len(set(y_pred)))

# evaluate
print('\tARI:', metrics.adjusted_rand_score(y_true, y_pred))
print('\tAdjust MI:', metrics.adjusted_mutual_info_score(y_true, y_pred))
print('\tv_measure_score:', metrics.v_measure_score(y_true, y_pred))

## Extract comments belongs to our concerned categories

In [39]:
from pyvi import ViTokenizer, ViPosTagger
import numpy as np
import sys
import gensim

In [ ]:
# load all documents
def iterdocuments(filenames):
    """
    Iterate over documents
    """
    index = 0
    for filename in filenames:
        df = pd.read_csv(filename, header=None, names=['text'])
        for _, row in df.iterrows():
            yield row['text']

corpus_paths = ['../data/%s_train.csv' % l for l in ['Positive', 'Neutral', 'Negative']]
documents = list(iterdocuments(corpus_paths))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
max_ngram = 3
vectorizer = CountVectorizer(ngram_range=(2, max_ngram))
analyze = vectorizer.build_analyzer()
analyze("This is a text document to analyze.")

In [41]:
file_path = '../data/features_dien-thoai-may-tinh-bang_affinity_token_set_ratio.csv'
df = pd.read_csv(file_path)
group_features = list()
for index, row in df.iterrows():
    group_features.append(set([x.strip("'. ") for x in row["similar_features"][1:-1].split(',') if len(x.strip("'. ")) > 0]))

In [42]:
df.head()

,similar_features
0,"['Chất_liệu Silicone_siêu_bền', 'Chất_liệu sil..."
1,"['Hỗ_trợ 3D , HDCP_Compliant', 'Hỗ_trợ HĐH iOS..."
2,"['Số ngõ ra', 'Số cổng ra', 'Ngõ ra', 'Loại cổ..."
3,"['Tốc_độ đọc', 'Tốc_độ đọc ( Mb )', 'Tốc_độ', ..."
4,"['2 Nano SIM', '2 Sim 2 Sóng', 'Sim 4G nghe và..."


In [ ]:
selected_feature = group_features[10]
tokenized_feature = [ViTokenizer.tokenize(f) for f in selected_feature]
print(tokenized_feature)

In [ ]:
selected_feature = ['Nghe nhạc Mp3', 'Nghe nhạc', 'nghe nhạc MP3', 'nghe nhạc loa ngoài']
tokenized_feature = [ViTokenizer.tokenize(f) for f in selected_feature]
print(tokenized_feature)

# set max ngrams
max_ngram = sorted([len(f.split()) for f in tokenized_feature], reverse=True)[0]
max_ngram = 3 if max_ngram < 3 else max_ngram
print('max_ngram:', max_ngram)
vectorizer = CountVectorizer(ngram_range=(2, max_ngram))
analyze = vectorizer.build_analyzer()

# find matching document
filter_documents = []
for index, d in enumerate(documents):
    sys.stdout.write('\r')
    sys.stdout.write('index: %s, filter_documents: %s' % (index, len(filter_documents)))
    sys.stdout.flush()
    terms = analyze(d)
    for term in terms:
        score = [0.]
        for feature in selected_feature:
            score.append(fuzz.token_set_ratio(term, feature))
        if np.average(score) >= 60:
            filter_documents.append(d)
            break


In [ ]:
filter_documents[:5]

In [ ]:
df = pd.DataFrame.from_dict([{'text': v} for v in filter_documents])
file_path = '../data/documents_related_%s.csv' % ('nghe_nhac')
df.to_csv(file_path, index=False)